# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## Using less dataset to train the model
Training dataset - 1200   
Test dataset - 240  
훈련 안 됨

## 필요한 library 호출

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2

import os

tf.logging.set_verbosity(tf.logging.INFO)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## CNN model 설계
현재는 MNIST에 사용된 것으로, 추후 그물 찢김에 적용하기 위해 수정 필요

### Input Layer
Input Image Pixel: 28x28  
Color channel: 1 (grayscale)  

### 1st Conv Layer
Num of filter: 32  
Kernel size: 5x5  
Padding: Same  
Activation: ReLU  
Padding same은 입/출력이 같은 크기를 갖도록 padding하는 것  

### 1st Pooling Layer
Max pooling  
Size: 2x2  
Stride: 2  
Max pooling 시 size의 폭과 stride가 같으면 pooling window가 서로 겹치지 않음  

### 2nd Conv Layer
Num of filter: 64  
Kernel size: 5x5  
Padding: Same  
Activation: ReLU  

### 2nd Pooling Layer
Max pooling  
Size: 2x2  
Stride: 2  


### 3rd, 4th, 5th


### Dense Layer
Fully connected  

### Drop out
Rate: 0.4 (40%)  
훈련 중 40%의 connection이 임의로 제거됨  
Overfitting 방지  
뇌의학에서도 학습을 반복할수록 사람 뇌의 뉴런은 일부 connection이 끊어지면서 더 확실하게 학습한다고 함  

### Logits Layer
Binaray classification  

### Training
Loss function: Softmax Cross Entropy  
Optimizer: Gradient Descent  
Learning rate: 0.001  

In [2]:
def cnn_model_fn(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 480, 640, 3])

    # Convolutional Layer #1
    # Computes 48 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 480, 640, 3]
    # Output Tensor Shape: [batch_size, 480, 640, 48]
    conv1 = tf.layers.conv2d(
        
        inputs=input_layer,
        filters=48,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    print(conv1.shape)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 480, 640, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 48]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    print(pool1.shape)

    # Convolutional Layer #2
    # Computes 96 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 240, 320, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 96]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    print(conv2.shape)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 240, 320, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    print(pool2.shape)

    
    
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    print(conv3.shape)
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    print(pool3.shape)
    
    
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]    
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    print(conv4.shape)
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    print(pool4.shape)
    
    
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]    
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    print(conv5.shape)
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 15, 20, 96]
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2)
    print(pool5.shape)
    
    
    
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 15, 20, 96]
    # Output Tensor Shape: [batch_size, 15 * 20 * 96]
    pool5_flat = tf.reshape(pool5, [-1, 15 * 20 * 96])
    print(pool5_flat.shape)


    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 15 * 20 * 96]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool5_flat, units=1024, activation=tf.nn.relu)
    print(dense.shape)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 1]
    logits = tf.layers.dense(inputs=dropout, units=2)
    print(logits.shape)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # sparse_softmax_cross_entropy cannot use one-hot encoding
    
    #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Code Run


### Training
Batch size: 128  
Epoch: 5  
Shuffle: True  
Step: 20000

## Loading dataset
Load dataset and split them into training data and evaluation data

파손, 정상 비율 1:1??

In [3]:
# torn: 0-1484
# untorn: 0-1779
num_torn = 720
num_untorn = 720
total_pix = 480*640*3

torn_data = np.zeros((num_torn, total_pix), dtype=np.float32)
print(torn_data.shape)
untorn_data = np.zeros((num_untorn, total_pix), dtype=np.float32)

(720, 921600)


In [4]:
num_torn_max = 1485
num_untorn_max = 1780

torn_idx = np.random.choice(np.arange(num_torn_max), num_torn)
untorn_idx = np.random.choice(np.arange(num_untorn_max), num_untorn)
print(torn_idx)

np.random.shuffle(torn_idx)
np.random.shuffle(untorn_idx)

print(torn_idx)


np.savetxt('torn_1200.csv', torn_idx, delimiter=',')
np.savetxt('untorn_1200.csv', untorn_idx, delimiter=',')

[ 351  815  438  857  886  241  793  409  575   32   19 1482 1225  599
  829  861  837 1058 1422  235  350   94  514 1310 1138 1410  489  343
  565  873    3  958  608  255 1351 1288  392 1040 1478 1244  334  142
  187  224 1331  465  484 1323  841  961   52 1000  145 1440   93  462
  280 1220  821  602  869 1172 1435  996 1142  735   85 1210   17  546
  464 1346  734  649 1103  565   73 1192  409  607  933  821  283 1085
   97 1396  184 1413 1423  733 1260   18 1124  603  888   71 1055  684
  433  822  783   48  272 1208 1356  604 1375 1325 1484 1206    6  762
  140 1105  899 1131  191 1074  608 1271  381 1297  845 1250  960  561
  641  137 1245  310   11  221  252  651  937  462  540 1334  802  542
 1135  253  187  937  598 1321  873  759  672  279 1410 1398 1079  954
  653    7  808  378  239  785  183  180  555  899  398  600  502  510
   65  764  799  934  774 1212  503  387  847   39 1040  183  197 1401
  770 1357  447 1347  210  183 1207 1167  862  876  723  857 1066  515
  225 

In [5]:
for i in range(num_torn):
    img = cv2.imread("/dataset/torn/img-%04d.png" % torn_idx[i]).flatten()
    torn_data[i][:] = img

In [6]:
for i in range(num_untorn):
    img = cv2.imread("/dataset/untorn/img-%04d.png" % untorn_idx[i]).flatten()
    untorn_data[i][:] = img

In [7]:
torn_label = np.full((num_torn,1), 1, dtype=np.float32)
untorn_label = np.full((num_untorn,1), 0, dtype=np.float32)

In [8]:
# 80% training, 20% evaluation
tr_rate = 600/720
num_tr_torn = int(tr_rate * num_torn)
num_ev_torn = num_torn - num_tr_torn
print(num_tr_torn, num_ev_torn)

num_tr_untorn = int(tr_rate * num_untorn)
num_ev_untorn = num_untorn - num_tr_untorn
print(num_tr_untorn, num_ev_untorn)

tr_torn = torn_data[:num_tr_torn][:]
ev_torn = torn_data[num_tr_torn:][:]
print(tr_torn.shape)
print(ev_torn.shape)

tr_untorn = untorn_data[:num_tr_untorn][:]
ev_untorn = untorn_data[num_tr_untorn:][:]
print(tr_untorn.shape)
print(ev_untorn.shape)



tr_data = np.append(tr_torn, tr_untorn, axis=0)
print(tr_data.shape)
ev_data = np.append(ev_torn, ev_untorn, axis=0)
print(ev_data.shape)

tr_label = np.append(np.full((num_tr_torn), 1, dtype=np.int32), np.full((num_tr_untorn), 0, dtype=np.int32))
ev_label = np.append(np.full((num_ev_torn), 1, dtype=np.int32), np.full((num_ev_untorn), 0, dtype=np.int32))
print(tr_label.shape[0], ev_label.shape[0])

600 120
600 120
(600, 921600)
(120, 921600)
(600, 921600)
(120, 921600)
(1200, 921600)
(240, 921600)
1200 240


In [9]:
shuf1 = np.arange(tr_label.shape[0])
np.random.shuffle(shuf1)
print(shuf1)

shuf2 = np.arange(ev_label.shape[0])
np.random.shuffle(shuf2)
print(shuf2)

tr_data_sh = np.zeros((tr_data.shape), dtype=np.float32)
tr_label_sh = np.zeros((tr_label.shape), dtype=np.int32)
ev_data_sh = np.zeros((ev_data.shape), dtype=np.float32)
ev_label_sh = np.zeros((ev_label.shape), dtype=np.int32)


for i in range(tr_label.shape[0]):
    tr_data_sh[i][:] = tr_data[shuf1[i]][:]
    tr_label_sh[i] = tr_label[shuf1[i]]
    
for i in range (ev_label.shape[0]):
    ev_data_sh[i][:] = ev_data[shuf2[i]][:]
    ev_label_sh[i] = ev_label[shuf2[i]]

[ 475  265  262 ... 1101 1177 1186]
[ 87  47 106  96 105 178 232  22 223 138 142 218 200 128  69 104  52  34
 227 137  71 195   7 139 148 117 145  60  40  66  23 231 111  46  91  64
 211  24 184  59 131 193  36 171 183 209   1 188  53 156  26  94 210   6
 166  81  31 110 155 192 114 208 153 173  93  85  82 120 217 170  89 150
  27 161 194 189  95  75 100  51 205  57 226 175 118 126  21 108 159 115
 235  16  20  76  33 234  80 151 214 199   2  43  14 122  92  88 164  72
  63   3  54 143 201  74 186  35 221  90 185 134  11 127 207  19  41 212
  68 133  78  84 220  12  37  79 129 165 233 182   9  28  99  65  30 123
  98 130 204  13 144   0 239 121 141 229 162   5 172 213   8  44 140 215
 132 158 157  58 147  62 202  50  15 167 198  10 107  17 222 176  29 174
  70 191 225  61  73  45 230 228 152 203 197 149 136 187 237  67 179 238
  32 180 109   4  25 206  83 196 236 102 177  55 146  42 124 160 119 163
  48 101  39  56 168  18 224 135  86 216  38 219 103 169 125 112 190  49
 181  77 113  9

In [11]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)

# Create the Estimator
net_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/models/CNN1200")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

batch_s = 1
epochs = 5
tr_steps = int(tr_data.shape[0]/batch_s*epochs)
in_steps = 1

for d in ['/device:GPU:0', '/device:GPU:1']:
    with tf.device(d):

        # Train the model
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": tr_data},
            y=tr_label,
            batch_size=batch_s,
            num_epochs=epochs,
            shuffle=True)
        net_classifier.train(
            input_fn=train_input_fn,
            steps=20000,
            hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0ad006ae10>, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_evaluation_master': '', '_is_chief': True, '_model_dir': '/models/CNN1200', '_save_checkpoints_steps': None, '_task_id': 0, '_session_config': None, '_service': None, '_task_type': 'worker', '_num_ps_replicas': 0, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None}
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:

INFO:tensorflow:probabilities = [[0.5029931 0.4970069]] (1.451 sec)
INFO:tensorflow:loss = 0.68717885, step = 3001 (2.894 sec)
INFO:tensorflow:probabilities = [[0.5063621  0.49363795]] (1.433 sec)
INFO:tensorflow:global_step/sec: 34.7044
INFO:tensorflow:probabilities = [[0.50866413 0.49133584]] (1.449 sec)
INFO:tensorflow:loss = 0.7106274, step = 3101 (2.881 sec)
INFO:tensorflow:probabilities = [[0.50696284 0.49303716]] (1.393 sec)
INFO:tensorflow:global_step/sec: 35.8903
INFO:tensorflow:probabilities = [[0.50875735 0.49124262]] (1.393 sec)
INFO:tensorflow:loss = 0.6757841, step = 3201 (2.786 sec)
INFO:tensorflow:probabilities = [[0.50950336 0.49049664]] (1.418 sec)
INFO:tensorflow:global_step/sec: 35.054
INFO:tensorflow:probabilities = [[0.5107272 0.4892728]] (1.435 sec)
INFO:tensorflow:loss = 0.71483505, step = 3301 (2.853 sec)
INFO:tensorflow:probabilities = [[0.5114415  0.48855847]] (1.423 sec)
INFO:tensorflow:global_step/sec: 35.0082
INFO:tensorflow:probabilities = [[0.51116246 0.

INFO:tensorflow:global_step/sec: 34.2299
INFO:tensorflow:probabilities = [[0.49510652 0.5048935 ]] (1.405 sec)
INFO:tensorflow:loss = 0.7029823, step = 6201 (2.922 sec)
INFO:tensorflow:probabilities = [[0.49473327 0.5052667 ]] (1.379 sec)
INFO:tensorflow:global_step/sec: 35.5651
INFO:tensorflow:probabilities = [[0.49138334 0.5086167 ]] (1.433 sec)
INFO:tensorflow:loss = 0.6760607, step = 6301 (2.811 sec)
INFO:tensorflow:probabilities = [[0.49011308 0.50988686]] (1.432 sec)
INFO:tensorflow:global_step/sec: 35.2719
INFO:tensorflow:probabilities = [[0.4893799  0.51062006]] (1.403 sec)
INFO:tensorflow:loss = 0.7146161, step = 6401 (2.836 sec)
INFO:tensorflow:probabilities = [[0.48765543 0.5123446 ]] (1.465 sec)
INFO:tensorflow:global_step/sec: 34.8592
INFO:tensorflow:probabilities = [[0.48844165 0.5115583 ]] (1.403 sec)
INFO:tensorflow:loss = 0.6702937, step = 6501 (2.868 sec)
INFO:tensorflow:probabilities = [[0.49022985 0.5097702 ]] (1.427 sec)
INFO:tensorflow:global_step/sec: 34.3627
INF

INFO:tensorflow:loss = 0.7391581, step = 9601 (2.795 sec)
INFO:tensorflow:probabilities = [[0.5214422  0.47855785]] (1.449 sec)
INFO:tensorflow:global_step/sec: 34.5257
INFO:tensorflow:probabilities = [[0.52089864 0.47910136]] (1.448 sec)
INFO:tensorflow:loss = 0.6521998, step = 9701 (2.897 sec)
INFO:tensorflow:probabilities = [[0.519379 0.480621]] (1.440 sec)
INFO:tensorflow:global_step/sec: 34.935
INFO:tensorflow:probabilities = [[0.5144546  0.48554546]] (1.422 sec)
INFO:tensorflow:loss = 0.7224824, step = 9801 (2.861 sec)
INFO:tensorflow:probabilities = [[0.509662   0.49033806]] (1.410 sec)
INFO:tensorflow:global_step/sec: 34.3926
INFO:tensorflow:probabilities = [[0.5059652  0.49403486]] (1.498 sec)
INFO:tensorflow:loss = 0.7051492, step = 9901 (2.907 sec)
INFO:tensorflow:probabilities = [[0.5008782  0.49912176]] (1.555 sec)
INFO:tensorflow:global_step/sec: 32.2853
INFO:tensorflow:probabilities = [[0.49839187 0.50160813]] (1.543 sec)
INFO:tensorflow:loss = 0.68993604, step = 10001 (

In [12]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
session = tf.Session(config=config)

ev_results = np.zeros((len(ev_label), 1))
with tf.device('/device:GPU:2'):

    # Evaluate the model and print results
    for i in range(len(ev_label)):
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": ev_data_sh[i:i+1]},
            y=ev_label_sh[i:i+1],
            num_epochs=1,
            shuffle=False)
        eval_results = net_classifier.evaluate(input_fn=eval_input_fn)
        
        ev_results[i] = eval_results["accuracy"]

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:17:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:17:01
INFO:tensorflow:Saving dict for global step 12000: accuracy = 1.0, global_step = 12000, loss = 0.63405806
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:17:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:17:06
INFO:tensorflow:Saving dict for global step 12000: accuracy = 0.0, global_step = 12000, loss = 0.7559481
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:17:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:17:11
INFO:tensorflow:Saving dict for global step 12000: accuracy = 0.0, global_step = 12000, loss = 0.7559481
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:17:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:17:16
INFO:tensorflow:Saving dict for global step 12000: accuracy = 0.0, global_step = 12000, loss = 0.7559481
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:17:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:17:21
INFO:tensorflow:Saving dict for global step 12000: accuracy = 1.0, global_step = 12000, loss = 0.63405806
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 

...

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:18:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:18:15
INFO:tensorflow:Saving dict for global step 12000: accuracy = 1.0, global_step = 12000, loss = 0.63405806
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:18:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:18:19
INFO:tensorflow:Saving dict for global step 12000: accuracy = 0.0, global_step = 12000, loss = 0.7559481
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:18:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:18:24
INFO:tensorflow:Saving dict for global step 12000: accuracy = 1.0, global_step = 12000, loss = 0.63405806
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:18:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:18:29
INFO:tensorflow:Saving dict for global step 12000: accuracy = 1.0, global_step = 12000, loss = 0.63405806
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 

INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-07:18:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/CNN1200/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-07:18:34
INFO:tensorflow:Saving dict for global step 12000: accuracy = 1.0, global_step = 12000, loss = 0.63405806
INFO:tensorflow:Calling model_fn.
(?, 480, 640, 48)
(?, 240, 320, 48)
(?, 240, 320, 96)
(?, 120, 160, 96)
(?, 120, 160, 96)
(?, 60, 80, 96)
(?, 60, 80, 96)
(?, 30, 40, 96)
(?, 30, 40, 96)
(?, 15, 20, 96)
(?, 28800)
(?, 1024)
(?, 2)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 

## Results
Training step 6000: 50%

In [13]:
print(np.mean(ev_results))

0.5


In [ ]:
fail_idx = np.where(ev_results == 0)
print(fail_idx[0])
print(len(fail_idx[0]))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


for i in range(len(fail_idx[0])):

    temp_img = ev_data_sh[fail_idx[0][i]][:].reshape([480, 640, 3])

    temp_img = 255-temp_img

    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.axis("off")
    plt.imshow(cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB))
    plt.show()

## Results
훈련 데이터셋 2612개  
검증 데이터셋 653개  
  
정확도 87.7%  
  
CNN layer 5개  
Training step: 40000회  
Batch size: 1  
